In [18]:
!pip install -U scikit-learn==0.24

Requirement already up-to-date: scikit-learn==0.24 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (0.24.0)


### References:

https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-advanced-entry-script

In [30]:
import sklearn
import pandas
from azureml.core import Model
from azureml.core.workspace import Workspace
from azureml.core.resource_configuration import ResourceConfiguration

ws = Workspace.from_config()

In [20]:
model = Model(workspace=ws, name="usedcarprice_gbm")

In [21]:
model

Model(workspace=Workspace.create(name='mlops2021', subscription_id='fbb085eb-3abd-4384-93aa-608f78277ad2', resource_group='mlopsdemo'), name=usedcarprice_gbm, id=usedcarprice_gbm:2, version=2, tags={'area': 'usedcar', 'type': 'regression'}, properties={})

In [31]:
import sklearn

from azureml.core.environment import Environment

environment = Environment("LocalDeploy")
environment.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
environment.python.conda_dependencies.add_pip_package("joblib")
environment.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))
environment.python.conda_dependencies.add_pip_package("pandas=={}".format(pandas.__version__))

In [32]:
%%writefile score.py
import os
import logging
import json
import numpy as np
import joblib
import pandas as pd
from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.standard_py_parameter_type import StandardPythonParameterType
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


def init():
    """
    This function is called when the container is initialized/started, typically after create/update of the deployment.
    You can write the logic here to perform init operations like caching the model in memory
    """
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    model_path = os.path.join(
        os.getenv("AZUREML_MODEL_DIR"), "carmodel.pkl"
    )
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    logging.info("Init complete")


pandas_sample_input = PandasParameterType(pd.DataFrame({ 'KM_Driven':[10],'Fuel_Type':['Petrol'],'age':[4],'Transmission':['Automatic'],'Owner_Type':['First'],'Seats':[4],'make':['maruti'],'mileage_new':[21],'engine_new':[900],'model':['swift'],'power_new':[90],'Location':['Bangalore']}))
output_sample = np.array([5.2])

@input_schema('data', pandas_sample_input)
# 'Inputs' is case sensitive
@output_schema(NumpyParameterType(output_sample))

def run(data):
    """
    This function is called for every invocation of the endpoint to perform the actual scoring/prediction.
    In the example we extract the data from the json input and call the scikit-learn model's predict()
    method and return the result back
    """
    logging.info("Request received")
    result = model.predict(data)
    logging.info("Request processed")
    return result.tolist()

Overwriting score.py


In [33]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=environment)

In [35]:
from azureml.core.webservice import LocalWebservice

service_name = 'usedcarprice-service'

local_service = Model.deploy(ws, service_name, [model], inference_config)

local_service.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"


In [38]:
import json

data = [{"KM_Driven": 10, "Fuel_Type": "Petrol", "age": 4, "Transmission": "Automatic", "Owner_Type": "First", "Seats": 4, "make": "maruti", "mileage_new": 21, "engine_new": 900, "model": "swift", "power_new": 90, "Location": "Bangalore"}]

input_payload = json.dumps({
    "data": data
})

output = local_service.run(input_payload)

print(output)

[7.385208175899092]
